In [1]:
import pandas as pd
import numpy as np
import os
from sklearn.metrics import log_loss

In [2]:
class Config:
    DATA_DIR = './data'
    OUTPUT_DIR = './outputs'

In [3]:
def get_weighted_gm(vers=[], weights=[]):

    prod = (vers[0][fts] * 0)+1
    for w, ver in zip(weights, vers):
        prod *= ver[fts] ** w
    return prod

In [4]:
def read_ver(ver_path, test=False):

    tmp_df = pd.concat([pd.read_csv(os.path.join(ver_path, c)) for c in os.listdir(ver_path) if 'val' in c])
    tmp_df = tmp_df.sort_values(by='img_IDS').reset_index(drop=True)

    if test:
        tmp_df = pd.concat([pd.read_csv(os.path.join(ver_path, c)) for c in os.listdir(ver_path) if 'test_preds_fold_' in c])
        fts = tmp_df.columns[1:]
        tmp_df = tmp_df.groupby('img_IDS', as_index=False)[fts].apply(lambda x: np.exp(np.mean(np.log(x)))) 
        tmp_df = tmp_df.sort_values(by='img_IDS').reset_index(drop=True)
    return tmp_df

In [5]:
v7 = read_ver(os.path.join(Config.OUTPUT_DIR, 'swin_v7'), test=True)
v10 = read_ver(os.path.join(Config.OUTPUT_DIR, 'swin_10'), test=True)
v11 = read_ver(os.path.join(Config.OUTPUT_DIR, 'swin_11'), test=True)
conv_v5 = read_ver(os.path.join(Config.OUTPUT_DIR, 'conv_v5'), test=True)
conv_v7 = read_ver(os.path.join(Config.OUTPUT_DIR, 'conv_v7'), test=True)
eff_v1 = read_ver(os.path.join(Config.OUTPUT_DIR, 'effnet_v1'), test=True)

blnd = v7.copy()
weights = [0.13, 0.12, 0.1,  0.30,    0.2,      0.15]
vers =    [v7,   v10,  v11,  conv_v5, conv_v7, eff_v1]
print(np.sum(weights))

fts = list(v7.columns[1:])
blnd[fts] = get_weighted_gm(vers=vers, weights=weights)
blnd.to_csv('blnd.csv', index=False)

0.9999999999999999
